In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
import re
from textstat.textstat import textstatistics

In [3]:
xl_file = pd.read_excel("Input.xlsx", )

In [3]:
for x in xl_file.index:
    url = xl_file["URL"][x]
    urlid = xl_file["URL_ID"][x]
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
        }
    r = requests.get(url, headers=headers)
    soup = bs(r.content, 'html5lib')
    # print(soup.prettify())
    mydivs = soup.find_all("h1", {"class": "entry-title"})
    mydivs += soup.find_all("div", {"class": "td-post-content"})
    
    for mydiv in mydivs:
        text = mydiv.text
    
    f = open("{}.txt".format(urlid), "x",encoding="utf-8")
    f.write(text)
    
    f.close()

In [4]:

pos = open("positive-words.txt", "r")
neg = open("negative-words.txt", "r")
positive_words = pos.read()
negative_words = neg.read()

columns_name = ["URL_ID", "URL","POSITIVE SCORE","NEAGTIVE SCORE","POLARITY SCORE","SUBJECTIVITY SCORE","AVG SENTENCE LENGTH","PERCENTAGE OF COMPLEX WORDS","FOG INDEX","AVG NUMBER OF WORDS PER SENTENCE","COMPLEX WORD COUNT","WORD COUNT","SYLLABLE PER WORD","PERSONAL PRONOUNS","AVG WORD LENGTH"]
final_df = pd.DataFrame(columns = columns_name)

lemmatizer = WordNetLemmatizer()

In [5]:
for x in xl_file.index:
    url = xl_file["URL"][x]
    urlid = xl_file["URL_ID"][x]
    
    f = open('{}.txt'.format(urlid), "r", encoding="utf8")
    content = f.read()
    
    # lemmatization and cleanning
    sentence = sent_tokenize(content)
    corp = str(content).lower() 
    corp = re.sub('[^a-zA-Z]+',' ', corp).strip() 
    tokens = word_tokenize(corp)
    words = [t for t in tokens if t not in set(stopwords.words("english"))]
    lemmatize = [lemmatizer.lemmatize(w) for w in words]
    total_clean_words = len(lemmatize)
    
    positive_score = 0
    negative_score = 0
    
    for i in lemmatize:
        if i in positive_words :
            positive_score += 1
        if i in negative_words :
            negative_score += -1

    # positive and negative score
    positive_score
    negative_score = negative_score * -1
    
    # polarity score
    polarity_score =  (positive_score - negative_score)/((positive_score + negative_score) + 0.000001)
    
    #subjective score
    subjective_score = (positive_score + negative_score)/ ((total_clean_words) + 0.000001)
    
    #average sentence length
    average_sentence_length = len(words)/len(sentence) 
    
    #average word per sentence
    average_word_per_sentence = len(words)/len(sentence) 
    
    #complex word count
    complex_words = set()
    for word in lemmatize:
        syllable_count = textstatistics().syllable_count(word)
        if syllable_count >= 2:
            complex_words.add(word)
    complex_word_count = len(complex_words)
    
    #percent of complex word
    percent_complex_words = len(complex_words) / len(lemmatize)
    
    #fog index
    fog_index = 0.4 * (average_sentence_length + percent_complex_words)
    
    #average syllable per word
    syllable_count = textstatistics().syllable_count(word)
    ASPW = syllable_count / len(lemmatize)
    
    # personal pronouns
    pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
    pronouns = pronounRegex.findall(content)
    personal_pronouns = len(pronouns)
    
    #average word length
    average_word_length = sum(len(word) for word in lemmatize) / len(lemmatize)
    
    new_row = {"URL_ID":urlid, "URL":url,"POSITIVE SCORE":positive_score,"NEAGTIVE SCORE":negative_score,"POLARITY SCORE":polarity_score,
               "SUBJECTIVITY SCORE":subjective_score,"AVG SENTENCE LENGTH":average_sentence_length,"PERCENTAGE OF COMPLEX WORDS":percent_complex_words,
               "FOG INDEX":fog_index,"AVG NUMBER OF WORDS PER SENTENCE":average_word_per_sentence,"COMPLEX WORD COUNT":complex_word_count,
               "WORD COUNT":total_clean_words,"SYLLABLE PER WORD":ASPW,"PERSONAL PRONOUNS":personal_pronouns,"AVG WORD LENGTH":average_word_length
              }
    final_df = final_df.append(new_row, ignore_index=True)


In [6]:
final_df.set_index('URL_ID', inplace=True)


In [8]:
# determining the name of the file
file_name = 'final_output_01.xlsx'
  
# saving the excel
final_df.to_excel(file_name)